In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
calendar = pd.read_csv("../input/m5-forecasting-accuracy/calendar.csv")
sales_train_validation = pd.read_csv("../input/m5-forecasting-accuracy/sales_train_validation.csv")
sample_submission = pd.read_csv("../input/m5-forecasting-accuracy/sample_submission.csv")
sell_prices = pd.read_csv("../input/m5-forecasting-accuracy/sell_prices.csv")

In [ ]:
sell_prices.head()

In [ ]:
sell_prices.shape

In [ ]:

sales_train_validation.shape

In [ ]:
sales_train_validation.head()

In [ ]:
calendar.shape

In [ ]:
calendar.head()

In [ ]:
sales_train_validation

In [ ]:
df = sales_train_validation
df = df[df['id']=='FOODS_3_826_WI_3_validation']

In [ ]:
df

In [ ]:
d_cols = [col for col in sales_train_validation.columns if 'd_' in col]

In [ ]:
example = df[d_cols].T
example = example.rename(columns={30488:'FOODS_3_826_WI_3'})
example

In [ ]:
example = example.reset_index().rename(columns={'index': 'd'})

In [ ]:
example = example.merge(calendar, how='left', validate='1:1')

In [ ]:
calendar

In [ ]:
example.head()

In [ ]:
sam = pd.concat([example.FOODS_3_826_WI_3, example.FOODS_3_826_WI_3.shift(-28)], axis=1)

In [ ]:
sam

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [ ]:
example.columns

In [ ]:
example = example.filter(['FOODS_3_826_WI_3','wday','month','year','snap_CA','snap_TX','snap_WI'])

In [ ]:


X = example[:-28]
X

In [ ]:
X.columns

In [ ]:
y = example.FOODS_3_826_WI_3.shift(-28).dropna()

In [ ]:
y

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler().fit(X)
df_rescaled = scaler.transform(X)

In [ ]:
df_rescaled

In [ ]:
X.iloc[:,0]

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(df_rescaled, y, random_state=42)

In [ ]:
rf_model = LogisticRegression()
rf_model.fit(X_train, y_train)

In [ ]:
rf_model.score(X_test, y_test)

In [ ]:
def scoring(data):
    d_cols = [col for col in data.columns if 'd_' in col]
    count=0
    total_score=[]
    for i in range(len(data)) : 
        id_name = data.loc[i, "id"]
        df = data[data['id']==id_name]
        example = df[d_cols].T
       
        example = example.rename(columns={count:id_name})

        example = example.reset_index().rename(columns={'index': 'd'})
        example = example.merge(calendar, how='left', validate='1:1')
        sam = pd.concat([example.iloc[:,-1], example.iloc[:,-1].shift(-28)], axis=1)
        example = example.filter([id_name,'wday','month','year','snap_CA','snap_TX','snap_WI'])
        count+=1
        X = example[:-28]
        y = example.iloc[:,0].shift(-28).dropna()
        scaler = StandardScaler().fit(X)
        df_rescaled = scaler.transform(X)
        X_train, X_test, y_train, y_test = train_test_split(df_rescaled, y, random_state=42)
        rf_model = SVC()
        rf_model.fit(X_train, y_train)
        score = rf_model.score(X_test, y_test)
        rf_model.score(X_test, y_test)
        print(score)
        total_score.append(score)
    avg_score = np.mean(total_score)
    print(avg_score)
        
    

In [ ]:
scoring(sales_train_validation)